In [61]:
import pandas as pd
acrplanted_pre = pd.read_csv('acrplanted_pre.csv')
acrplanted_pre.head()

,Program,Year,Ag District,Ag District Code,County,County ANSI,Commodity,Data Item,Domain,Domain Category,Value,CV (%),avg_precip_count,avg_precip_mean,avg_precip_std,avg_precip_min,avg_precip_25%,avg_precip_50%,avg_precip_75%,avg_precip_max
0,SURVEY,2019,CENTRAL,50,cayuga,11.0,CORN,CORN - ACRES PLANTED,TOTAL,NOT SPECIFIED,68000.0,NaN,52.0,0.126284,0.088166,0.011262,0.064633,0.108586,0.162758,0.364602
1,SURVEY,2019,CENTRAL,50,cortland,23.0,CORN,CORN - ACRES PLANTED,TOTAL,NOT SPECIFIED,16000.0,NaN,52.0,0.136549,0.094494,0.008701,0.052037,0.116943,0.184175,0.391079
2,SURVEY,2019,CENTRAL,50,herkimer,43.0,CORN,CORN - ACRES PLANTED,TOTAL,NOT SPECIFIED,15400.0,NaN,52.0,0.163455,0.104624,0.003057,0.089469,0.134099,0.207785,0.490518
3,SURVEY,2019,CENTRAL,50,madison,53.0,CORN,CORN - ACRES PLANTED,TOTAL,NOT SPECIFIED,26600.0,NaN,52.0,0.137838,0.091820,0.008370,0.069401,0.110289,0.185823,0.375661
4,SURVEY,2019,CENTRAL,50,oneida,65.0,CORN,CORN - ACRES PLANTED,TOTAL,NOT SPECIFIED,34700.0,NaN,52.0,0.166656,0.110897,0.005187,0.077669,0.146137,0.232315,0.479629


In [63]:

m1 = acrplanted_pre[['Value','avg_precip_mean']].copy()






In [64]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

m1_scaled = sc.fit_transform(m1)

m1_s = pd.DataFrame(m1_scaled, columns=m1.columns, index=m1.index)

m1_s



,Value,avg_precip_mean
0,2.724334,0.228274
1,-0.419355,0.745077
2,-0.455628,2.099666
3,0.221474,0.809943
4,0.711164,2.260804
...,...,...
714,0.275884,-1.607777
715,0.439114,-1.508240
716,0.481433,-1.161680
717,1.811455,-0.319068


In [65]:
# Define X and y 
X = m1_s[['avg_precip_mean']]
y = m1_s[['Value']]

In [66]:
from sklearn.model_selection import train_test_split
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state = 0 )

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state = 0 ) 

print(x_train, x_val, x_test)

     avg_precip_mean
109        -0.627543
700        -1.419823
101        -0.553833
499         0.295024
319        -0.177591
..               ...
707        -1.630033
192         0.508349
629         0.804528
559        -1.160927
684         0.028400

[539 rows x 1 columns]      avg_precip_mean
302        -0.255222
652        -0.526495
79          0.429299
175        -0.629032
489         1.363709
..               ...
542         0.735562
569         0.853857
487         1.349547
521         0.866684
625         0.719879

[108 rows x 1 columns]      avg_precip_mean
249        -1.682454
40          0.381562
293         1.560164
711        -2.166027
211        -1.405137
..               ...
306        -0.456047
390         0.852816
251        -1.306953
245        -0.241839
27         -1.487627

[72 rows x 1 columns]


Linear Regression - validation

In [67]:
# build the model 
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train, y_train)

y_pre = model.predict(x_val) 



In [68]:
import sklearn.metrics as metrics
def regression_results(y_val, y_pre):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_val, y_pre)
    mean_absolute_error=metrics.mean_absolute_error(y_val, y_pre) 
    mse=metrics.mean_squared_error(y_val, y_pre) 
    #mean_squared_log_error=metrics.mean_squared_log_error(y_val, y_pre)
    median_absolute_error=metrics.median_absolute_error(y_val, y_pre)
    r2=metrics.r2_score(y_val, y_pre)

    print('explained_variance: ', round(explained_variance,4))    
    #print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))



In [69]:
#metrics for validation set - Linear Regression
import numpy as np
regression_results(y_val, y_pre)

explained_variance:  0.1207
r2:  0.1177
MAE:  0.7549
MSE:  0.8265
RMSE:  0.9091


In [73]:
#predict for test
model_testLR = LinearRegression()
model_testLR.fit(x_train, y_train)

y_pred_test = model_testLR.predict(x_test)

In [74]:
#regression results test
def regression_results_test(y_test, y_pred_test):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_test, y_pred_test)
    mean_absolute_error=metrics.mean_absolute_error(y_test, y_pred_test) 
    mse=metrics.mean_squared_error(y_test, y_pred_test) 
    median_absolute_error=metrics.median_absolute_error(y_test, y_pred_test)
    r2=metrics.r2_score(y_test, y_pred_test)

    print('explained_variance: ', round(explained_variance,4))    
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))


In [75]:
#metrics for test set - linear regression
regression_results_test(y_test, y_pred_test)

explained_variance:  0.1626
r2:  0.159
MAE:  0.8654
MSE:  1.0936
RMSE:  1.0458


Multivariate regression

In [70]:
multi = acrplanted_pre[['Value','avg_precip_mean', 'avg_precip_25%', 'avg_precip_50%', 'avg_precip_75%']].copy()
multi_scaled = sc.fit_transform(multi)

multi_s = pd.DataFrame(multi_scaled, columns=multi.columns, index=multi.index)

multi_s

,Value,avg_precip_mean,avg_precip_25%,avg_precip_50%,avg_precip_75%
0,2.724334,0.228274,1.554543,0.577443,-0.349783
1,-0.419355,0.745077,0.662320,0.967046,0.343665
2,-0.455628,2.099666,3.313764,1.766916,1.108146
3,0.221474,0.809943,1.892331,0.656815,0.397051
4,0.711164,2.260804,2.477963,2.328125,1.902396
...,...,...,...,...,...
714,0.275884,-1.607777,-1.400714,-1.400538,-1.171437
715,0.439114,-1.508240,-1.709911,-1.667919,-1.252839
716,0.481433,-1.161680,-1.240564,-1.554080,-1.128580
717,1.811455,-0.319068,-0.278301,-0.783386,-0.291813


In [71]:
X=multi_s[['avg_precip_25%', 'avg_precip_50%', 'avg_precip_75%', 'avg_precip_mean']]
y = multi_s[["Value"]]
from sklearn.model_selection import train_test_split
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state = 0 )

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state = 0 ) 

print(x_train, x_val, x_test)

     avg_precip_25%  avg_precip_50%  avg_precip_75%  avg_precip_mean
109       -0.113551        0.313531       -0.459661        -0.627543
700       -1.472903       -1.570358       -1.085575        -1.419823
101        0.524682        0.003701       -1.270237        -0.553833
499        0.793803       -0.328277        1.177632         0.295024
319       -0.576164       -0.599852       -0.314405        -0.177591
..              ...             ...             ...              ...
707       -1.543820       -1.471870       -1.555746        -1.630033
192       -0.767756       -0.452540        0.505798         0.508349
629        0.471588        1.205543        0.588741         0.804528
559       -0.748723       -1.119590       -1.083233        -1.160927
684       -0.100803       -0.181476        0.323838         0.028400

[539 rows x 4 columns]      avg_precip_25%  avg_precip_50%  avg_precip_75%  avg_precip_mean
302        0.455271        0.189230       -0.360803        -0.255222
652       

In [72]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train, y_train)

y_pre = model.predict(x_val) 
import sklearn.metrics as metrics
def regression_results(y_val, y_pre):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_val, y_pre)
    mean_absolute_error=metrics.mean_absolute_error(y_val, y_pre) 
    mse=metrics.mean_squared_error(y_val, y_pre) 
    #mean_squared_log_error=metrics.mean_squared_log_error(y_val, y_pre)
    median_absolute_error=metrics.median_absolute_error(y_val, y_pre)
    r2=metrics.r2_score(y_val, y_pre)

    print('explained_variance: ', round(explained_variance,4))    
    #print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))
regression_results(y_val, y_pre)

explained_variance:  0.1465
r2:  0.14
MAE:  0.7419
MSE:  0.8056
RMSE:  0.8975


Random forest

In [76]:
X = m1_s[['avg_precip_mean']]
y = m1_s[['Value']]
from sklearn.model_selection import train_test_split
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state = 0 )

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state = 0 ) 

Random Forest Validation

In [77]:
from sklearn.ensemble import RandomForestRegressor
randomForest_valid = RandomForestRegressor()
randomForest_valid.fit(x_train, y_train)

y_pred_valid_RF = randomForest_valid.predict(x_val)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [78]:
regression_results(y_val, y_pred_valid_RF)

explained_variance:  -0.2197
r2:  -0.2297
MAE:  0.8653
MSE:  1.1519
RMSE:  1.0732


Random Forest Test Set

In [79]:
randomForest_valid = RandomForestRegressor()
randomForest_valid.fit(x_train, y_train)

y_pred_test_RF = randomForest_valid.predict(x_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [80]:
regression_results_test(y_test, y_pred_test_RF)

explained_variance:  -0.3928
r2:  -0.3943
MAE:  1.12
MSE:  1.8132
RMSE:  1.3466


Random Forest with Multiple x-variables

In [81]:
X = multi_s[['avg_precip_25%', 'avg_precip_50%', 'avg_precip_75%', 'avg_precip_mean']]
y = multi_s[['Value']]
from sklearn.model_selection import train_test_split
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state = 0 )

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state = 0 ) 
randomForest_valid = RandomForestRegressor()
randomForest_valid.fit(x_train, y_train)

y_pred_valid_RF = randomForest_valid.predict(x_val)

regression_results(y_val, y_pred_valid_RF)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


explained_variance:  0.1639
r2:  0.1553
MAE:  0.7204
MSE:  0.7913
RMSE:  0.8895


**yield precipitation dataset**

In [82]:
crop_yield = pd.read_csv("yield_pre.csv")
crop_yield.head()

,Program,Year,Ag District,Ag District Code,County,County ANSI,Commodity,Data Item,Domain,Domain Category,Value,CV (%),avg_precip_count,avg_precip_mean,avg_precip_std,avg_precip_min,avg_precip_25%,avg_precip_50%,avg_precip_75%,avg_precip_max
0,SURVEY,2019,CENTRAL,50,cayuga,11.0,CORN,CORN GRAIN - YIELD MEASURED IN BU / ACRE,TOTAL,NOT SPECIFIED,158.0,NaN,52.0,0.126284,0.088166,0.011262,0.064633,0.108586,0.162758,0.364602
1,SURVEY,2019,CENTRAL,50,cortland,23.0,CORN,CORN GRAIN - YIELD MEASURED IN BU / ACRE,TOTAL,NOT SPECIFIED,157.6,NaN,52.0,0.136549,0.094494,0.008701,0.052037,0.116943,0.184175,0.391079
2,SURVEY,2019,CENTRAL,50,herkimer,43.0,CORN,CORN GRAIN - YIELD MEASURED IN BU / ACRE,TOTAL,NOT SPECIFIED,147.9,NaN,52.0,0.163455,0.104624,0.003057,0.089469,0.134099,0.207785,0.490518
3,SURVEY,2019,CENTRAL,50,madison,53.0,CORN,CORN GRAIN - YIELD MEASURED IN BU / ACRE,TOTAL,NOT SPECIFIED,139.3,NaN,52.0,0.137838,0.091820,0.008370,0.069401,0.110289,0.185823,0.375661
4,SURVEY,2019,CENTRAL,50,oneida,65.0,CORN,CORN GRAIN - YIELD MEASURED IN BU / ACRE,TOTAL,NOT SPECIFIED,154.4,NaN,52.0,0.166656,0.110897,0.005187,0.077669,0.146137,0.232315,0.479629


In [83]:
m2 = crop_yield[['Value','avg_precip_mean']].copy()
sc = StandardScaler()

m2_scaled = sc.fit_transform(m2)

m2_s = pd.DataFrame(m2_scaled, columns=m2.columns, index=m2.index)

m2_s

X = m2_s[['avg_precip_mean']]
y = m2_s[['Value']]




In [84]:
from sklearn.model_selection import train_test_split
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state = 0 )

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state = 0 ) 

print(x_train, x_val, x_test)

     avg_precip_mean
109        -0.627543
700        -1.419823
101        -0.553833
499         0.295024
319        -0.177591
..               ...
707        -1.630033
192         0.508349
629         0.804528
559        -1.160927
684         0.028400

[539 rows x 1 columns]      avg_precip_mean
302        -0.255222
652        -0.526495
79          0.429299
175        -0.629032
489         1.363709
..               ...
542         0.735562
569         0.853857
487         1.349547
521         0.866684
625         0.719879

[108 rows x 1 columns]      avg_precip_mean
249        -1.682454
40          0.381562
293         1.560164
711        -2.166027
211        -1.405137
..               ...
306        -0.456047
390         0.852816
251        -1.306953
245        -0.241839
27         -1.487627

[72 rows x 1 columns]


Linear Regression

In [85]:
model_yield_LR = LinearRegression()
model_yield_LR.fit(x_train, y_train)

y_pre_yield = model_yield_LR.predict(x_val) 
regression_results(y_val, y_pre_yield)

explained_variance:  0.0141
r2:  0.007
MAE:  0.7639
MSE:  1.0019
RMSE:  1.0009


In [86]:
model_testLR_yield = LinearRegression()
model_testLR_yield.fit(x_train, y_train)

y_pred_test_yield = model_testLR_yield.predict(x_test)
regression_results_test(y_test, y_pred_test_yield)


explained_variance:  0.0151
r2:  0.0124
MAE:  0.9007
MSE:  1.5289
RMSE:  1.2365


Multivariable regression

In [87]:
m2_multi = crop_yield[['Value','avg_precip_mean', 'avg_precip_min',	'avg_precip_25%',	'avg_precip_50%',	'avg_precip_75%']].copy()
sc = StandardScaler()

m2_scaled_multi = sc.fit_transform(m2_multi)

m2_s_multi = pd.DataFrame(m2_scaled_multi, columns=m2_multi.columns, index=m2_multi.index)

m2_s_multi

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state = 0 )

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state = 0 ) 

print(x_train, x_val, x_test)

X = m2_s_multi[['avg_precip_mean', 'avg_precip_min',	'avg_precip_25%',	'avg_precip_50%',	'avg_precip_75%']]
y = m2_s_multi[['Value']]
model_yield_LR = LinearRegression()
model_yield_LR.fit(x_train, y_train)

y_pre_yield = model_yield_LR.predict(x_val) 
regression_results(y_val, y_pre_yield)



     avg_precip_mean
109        -0.627543
700        -1.419823
101        -0.553833
499         0.295024
319        -0.177591
..               ...
707        -1.630033
192         0.508349
629         0.804528
559        -1.160927
684         0.028400

[539 rows x 1 columns]      avg_precip_mean
302        -0.255222
652        -0.526495
79          0.429299
175        -0.629032
489         1.363709
..               ...
542         0.735562
569         0.853857
487         1.349547
521         0.866684
625         0.719879

[108 rows x 1 columns]      avg_precip_mean
249        -1.682454
40          0.381562
293         1.560164
711        -2.166027
211        -1.405137
..               ...
306        -0.456047
390         0.852816
251        -1.306953
245        -0.241839
27         -1.487627

[72 rows x 1 columns]
explained_variance:  0.0141
r2:  0.007
MAE:  0.7639
MSE:  1.0019
RMSE:  1.0009


Random Forest

In [88]:
randomForest_valid_yield = RandomForestRegressor()
randomForest_valid_yield.fit(x_train, y_train)

y_pred_valid_RF_yield = randomForest_valid_yield.predict(x_val)
regression_results(y_val, y_pred_valid_RF_yield)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


explained_variance:  -0.156
r2:  -0.1651
MAE:  0.8117
MSE:  1.1754
RMSE:  1.0842


In [89]:
randomForest_yield_test = RandomForestRegressor()
randomForest_yield_test.fit(x_train, y_train)

y_pred_test_RF_yield = randomForest_yield_test.predict(x_test)
regression_results_test(y_test, y_pred_test_RF_yield)

explained_variance:  -0.1342
r2:  -0.155
MAE:  0.9646
MSE:  1.7881
RMSE:  1.3372


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Random Forest with Multiple x variables

In [90]:
X = m2_s_multi[['avg_precip_mean', 'avg_precip_min',	'avg_precip_25%',	'avg_precip_50%',	'avg_precip_75%']]
y = m2_s_multi[['Value']]
randomForest_valid_yield = RandomForestRegressor()
randomForest_valid_yield.fit(x_train, y_train)

y_pred_valid_RF_yield = randomForest_valid_yield.predict(x_val)
regression_results(y_val, y_pred_valid_RF_yield)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


explained_variance:  -0.181
r2:  -0.1915
MAE:  0.8164
MSE:  1.2021
RMSE:  1.0964
